# Pictures compression using SVD
In this exercise you are supposed to study how SVD could be used in image compression.

_Based on open course in [Numerical Linear Algebra](https://github.com/oseledets/nla2018) by Ivan Oseledets_

In [ ]:
# If you are using colab, uncomment this cell

# ! wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/a5bf18c/datasets/waiting.jpeg
# ! wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/a5bf18c/datasets/mipt.jpg
# ! wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/a5bf18c/datasets/simpsons.jpg

# ! mkdir ../dataset
# ! mv -t ../dataset waiting.jpeg mipt.jpg simpsons.jpg

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

## 1. Singular values

Compute the singular values of some predownloaded image (via the code provided below) and plot them. Do not forget to use logarithmic scale.

In [ ]:
face_raw = Image.open("../dataset/waiting.jpeg")
face = np.array(face_raw).astype(np.uint8)

plt.imshow(face_raw)
plt.xticks(())
plt.yticks(())
plt.title("Original Picture")
plt.show()

In [ ]:
# Image is saved as a 3-dimensional array with shape H x W x C (heigt x width x channels)
Rf = face[:, :, 0]
Gf = face[:, :, 1]
Bf = face[:, :, 2]

# Compute SVD and plot the singular values for different image channels
u, Rs, vh = np.linalg.svd(Rf, full_matrices=False)
u, Gs, vh = np.linalg.svd(Gf, full_matrices=False)
u, Bs, vh = np.linalg.svd(Bf, full_matrices=False)

plt.figure(figsize=(12, 7))
plt.plot(Rs, "ro")
plt.plot(Gs, "g.")
plt.plot(Bs, "b:")
plt.yscale("log")
plt.ylabel("Singular vals")
plt.show()

## 2. Compress

Complete a function ```compress```, that performs SVD and truncates it (using $k$ singular values/vectors). See the prototype below. 

Note, that in case when your images are not grayscale you have to split your image to channels and work with matrices corresponding to different channels separately.

Plot approximate reconstructed image $M_\varepsilon$ of your favorite image such that $rank(M_\varepsilon) = 5, 20, 50$ using ```plt.subplots```.

In [ ]:
def compress(image: np.ndarray, k: int) -> (np.ndarray, np.ndarray):
    """
    Perform svd decomposition and truncate it (using k singular values/vectors)

    Parameters:
        image (np.array): input image (probably, colourful)
        k (int): approximation rank

    Returns:
      reconst_matrix (np.array): reconstructed matrix (tensor in colourful case)
      s (np.array): array of singular values
    """
    image2 = image.copy()

    Rf = image2[:, :, 0]
    Gf = image2[:, :, 1]
    Bf = image2[:, :, 2]

    # compute per-channel SVD for and reconstruct the input image with the given approximation rank
    Ru, Rs, Rvh = np.linalg.svd(Rf, full_matrices=False)
    Gu, Gs, Gvh = np.linalg.svd(Gf, full_matrices=False)
    Bu, Bs, Bvh = np.linalg.svd(Bf, full_matrices=False)
    Rs = Rs[:k]
    Gs = Gs[:k]
    Bs = Bs[:k]

    reduced_im = np.zeros((image.shape), np.uint8)
    Red = Ru[:, :k] @ np.diag(Rs) @ Rvh[:k, :]
    Green = Gu[:, :k] @ np.diag(Gs) @ Gvh[:k, :]
    Blue = Bu[:, :k] @ np.diag(Bs) @ Bvh[:k, :]
    reduced_im[:, :, 0] = Red
    reduced_im[:, :, 1] = Green
    reduced_im[:, :, 2] = Blue

    s = np.zeros((len(Gs), 3))
    s[:, 0] = Rs
    s[:, 1] = Gs
    s[:, 2] = Bs
    return reduced_im.copy(), s

In [ ]:
plt.figure(figsize=(18, 12))
for i, k in enumerate([350, 300, 250, 200, 150, 100, 50, 20, 5]):
    plt.subplot(3, 3, i + 1)
    im, s = compress(face, k)
    plt.imshow(Image.fromarray(im, "RGB"))
    plt.xticks(())
    plt.yticks(())
    plt.title(f"{k} greatest SV")

## 3. Discover

Plot the following two figures for your favorite picture
* How relative error of approximation depends on the rank of approximation?
* How compression rate in terms of storing information ((singular vectors + singular numbers) / total size of image) depends on the rank of approximation?

In [ ]:
def compute_relative_error(image: np.ndarray, compressed_image: np.ndarray) -> np.float64:
    image = image.astype(np.float64)
    compressed_image = compressed_image.astype(np.float64)

    # error = MSE(image, compressed_image) / l2_norm(image)
    relative_error = np.linalg.norm(image - compressed_image)
    relative_error /= np.linalg.norm(image)

    return relative_error

In [ ]:
# fancy progress bar
from tqdm.auto import tqdm


k_list = range(5, 386, 5)
rel_err = []
info = []
for k in tqdm(k_list, leave=False):
    img, _ = compress(face, k)

    current_relative_error = compute_relative_error(face, img)

    # U(385xK) @ S(diag KxK) @ V^T(Kx498)
    current_information = k * (385 + 1 + 498)

    rel_err.append(current_relative_error)
    info.append(current_information)

plt.figure(figsize=(12, 7))

plt.subplot(2, 1, 1)
plt.title("Memory volume plot")
plt.xlabel("Rank")
plt.ylabel("Bytes")
plt.plot(k_list, info)

plt.subplot(2, 1, 2)
plt.title("Relative error plot")
plt.xlabel("Rank")
plt.ylabel("Rel err value")
plt.plot(k_list, rel_err)

plt.tight_layout()
plt.show()

## 4. Compare

 Consider the following two pictures. Compute their approximations (with the same rank, or relative error). What do you see? Explain results.

In [ ]:
image_raw1 = Image.open("../dataset/mipt.jpg")
image_raw2 = Image.open("../dataset/simpsons.jpg")

image1 = np.array(image_raw1).astype(np.uint8)
image2 = np.array(image_raw2).astype(np.uint8)

plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.imshow(image_raw1)
plt.title("One Picture")
plt.xticks(())
plt.yticks(())

plt.subplot(1, 2, 2)
plt.imshow(image_raw2)
plt.title("Another Picture")
plt.xticks(())
plt.yticks(())

plt.show()

### Same Rank

In [ ]:
im1, _ = compress(image1, 100)
im2, _ = compress(image2, 100)

plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
plt.imshow(Image.fromarray(im1, "RGB"))
plt.xticks(())
plt.yticks(())

plt.subplot(1, 2, 2)
plt.imshow(Image.fromarray(im2, "RGB"))
plt.xticks(())
plt.yticks(())

plt.show()

### Same Relative Error

In [ ]:
k_list = range(5, 500, 1)
rel_err1 = []
rel_err2 = []
relative_error_threshold = 0.15

for k in tqdm(k_list):
    image1_compressed, s = compress(image1, k)
    image2_compressed, s = compress(image2, k)

    relative_error_1 = compute_relative_error(image1, image1_compressed)
    relative_error_2 = compute_relative_error(image2, image2_compressed)

    rel_err1.append(relative_error_1)
    rel_err2.append(relative_error_2)

# find the indices
idx1 = int(np.argwhere(np.diff(np.sign(np.array(rel_err1) - relative_error_threshold))).flatten())
idx2 = int(np.argwhere(np.diff(np.sign(np.array(rel_err2) - relative_error_threshold))).flatten())
print("K1 = {}; K2 = {}".format(k_list[idx1], k_list[idx2]))

plt.figure(figsize=(12, 7))

plt.plot(k_list[idx1], rel_err1[idx1], "ro")
plt.plot(k_list[idx2], rel_err2[idx2], "ro")
plt.title("Rel err for 2 pics")
plt.xlabel("Rank")
plt.ylabel("Rel error val")

plt.plot(k_list, rel_err1, label="Image 1")
plt.plot(k_list, rel_err2, label="Image 2")
plt.plot(k_list, [relative_error_threshold] * len(k_list), ":")

plt.legend()
plt.show()

In [ ]:
image1_compressed, _ = compress(image1, idx1)
image2_compressed, _ = compress(image2, idx2)

plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
plt.imshow(Image.fromarray(image1_compressed, "RGB"))
plt.xticks(())
plt.yticks(())

plt.subplot(1, 2, 2)
plt.imshow(Image.fromarray(image2_compressed, "RGB"))
plt.xticks(())
plt.yticks(())
plt.show()